In [1]:
#----- STRUCTURE GENERATOR -----
# 1. Generates random carbon structures
# 2. Writes them into ase structure files in Random_Carbon_Structures

# --- INPUTS ---
# Structure Parameters
number_of_atoms = 200
density_g_cm_3 = 2
minimum_interatomic_distance = 1.76
atomic_number = 6
atomic_mass = 12

# Random Seed
import numpy as np
rng = np.random.default_rng(1) 

# --- STRUCTURE GENERATOR ---
import ase 
from ase.io import write
import os

# Structure Generator
def random_cell_generator(number_of_atoms, density_g_cm_3, min_dist, rng):
    
    # Density calculator - ASE uses Angstrom
    number_density = density_g_cm_3 / atomic_mass * 6.022 * 10 ** 23 / (10**24)

    # Cell Properties
    cell_volume = number_of_atoms / number_density
    cell_length = cell_volume ** (1/3)
    cell = np.full(3, cell_length)

    # Identity and position of atoms
    numbers = np.full(number_of_atoms, atomic_number)
    positions = []
    
    # Writes Data into the directory "Random_Carbon_Structures"
    data_dir = "Random_Carbon_Structures"
    os.makedirs(data_dir, exist_ok=True)

    filename = (
    f"{number_of_atoms}atoms_"
    f"{density_g_cm_3:.1f}gcm_"
    f"{minimum_interatomic_distance}minsep.data"
)

    file_path = os.path.join(data_dir, filename)

    # Check if structure already exists
    if os.path.exists(file_path):
        return None
    
    # Sanity Check on density
    mean_separation = (1 / number_density) ** (1/3)
    if min_dist > 0.99*mean_separation:
        return(print(f"FAILED: Mininum distance too close to mean separation of {mean_separation}"))
    if min_dist > 0.98*mean_separation:
        print(f"WARNING - SLOW:Minimum distance approaching mean separation of {mean_separation}")
    
    # Rejection Sampling used to generate random positions
    while len(positions) < number_of_atoms:   
        new_atom = rng.random(3) * cell_length
        if len(positions) == 0:
                nearest_neighbour = np.inf
        else:  
            pos_array = np.array(positions)
            vector_diffs = pos_array - new_atom
            scalar_diffs = np.linalg.norm(vector_diffs, axis=1)
            nearest_neighbour = min(scalar_diffs)
        if nearest_neighbour > min_dist:
            positions.append(new_atom)
            
    # Final Structure
    structure = ase.Atoms(
        numbers=numbers,
        positions=positions,
        cell=cell,
        pbc=True
    )

    write(
        file_path,
        structure,
        format="lammps-data",
        atom_style="atomic" 
    )
    return file_path

random_cell_generator(number_of_atoms, density_g_cm_3, minimum_interatomic_distance, rng )

for density_g_cm_3 in np.arange (1.5,3.6,0.1):
    new_file = random_cell_generator(number_of_atoms, density_g_cm_3, minimum_interatomic_distance, rng )
    if new_file is None:
        continue
    print(f"{new_file} created")
